In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import pickle
import scipy as sp
from skimage import data, io, filters
from skimage.color import rgb2hsv, hsv2rgb, xyz2rgb
from skimage import data, img_as_float
from skimage import exposure

from lippmann import show_spectrum, show_lippmann_transform, lippmann_transform
from display_spectral_data import load_specim_data
from color_tools import upsample_hue_saturation, from_spectrum_to_xyz, from_xyz_to_rgb
from spectrum import Spectrum3D
import spectrum_recovery_pool as recov

### Warning: data has to be downloaded from SWITCH drive (shared by Arnaud)

In [ ]:
name = "color_checker" # prefix of the file with data and results 

directory = "color_checker_high_res"

results_path = f"PNAS/{directory}/{name}"

c0 = 299792458
n0 = 1.5
c = c0 / n0

result = np.load(f"{results_path}.npy")
negative_rato = np.min(result)/np.max(result)
if negative_rato < 0:
    print(f"there are negative values in power spectrum of magnitude {negative_rato}")
result = np.clip(result, a_min = 0, a_max = np.inf)
with open(f"{results_path}.pkl", "rb") as pickled_params:
    params = pickle.load(pickled_params)

if "downsampling" in params:
    ds = params["downsampling"]
else:
    ds = 25
    estimate_decay
# if the data is stored in "Cubes"     
downsampled, wavelengths = load_specim_data("Cubes_ours/" + name, 1, cut=True)

omegas_rec = 2 * np.pi * c / wavelengths
if params["visible"]: #TODO what should be there? 
    omegas_est = np.linspace(2 * np.pi * c / 400E-9, 2 * np.pi * c / 700E-9, result.shape[2])
    print("Restricting spectrum to visible")
else:   
    omegas_est = np.linspace(np.max(omegas_rec), np.min(omegas_rec), result.shape[2])
wavelengths_est = 2 * np.pi * c / omegas_est

### Display experiment parameters and errors

In [ ]:
print("Experiment setup")
print_params = params.copy()

Z = params["Z"]
k0 = params["k0"]

if "estimate_depth" in params:
    if params["estimate_depth"]:
        plt.matshow(params["Z_estimates"])
        plt.colorbar()
        plt.savefig(f"{results_path}_z.pdf")
        plt.show()
        print_params.pop("Z_estimates")
        print_params.pop("k0_estimates")
        Z = np.median(params['Z_estimates'])
        k0 = np.median(params["k0_estimates"])
        print(f"median depth {Z:.2e}")
        print(f"median decay {k0}")
print_params.pop("errors")
print_params

In [ ]:
plt.matshow(params["errors"])
plt.colorbar()
plt.show()

### Convert everything to RGB and choose pixel positions

In [ ]:
pixel = (12, 12)
pixel_true = (11, 10)

ground_truth, _ = load_specim_data("Cubes_ours/" + name + "_true", params["downsampling"], cut=True)

recorded_xyz = from_spectrum_to_xyz(wavelengths, downsampled, normalize=False)
recorded_xyz = recorded_xyz / np.min(np.sum(recorded_xyz, axis=2))
recorded_rgb = from_xyz_to_rgb(recorded_xyz)

estimated_xyz = from_spectrum_to_xyz(wavelengths_est, result, normalize=False)
estimated_xyz = estimated_xyz / np.min(np.sum(estimated_xyz, axis=2))
estimated_rgb = from_xyz_to_rgb(estimated_xyz)

ground_truth_xyz = from_spectrum_to_xyz(wavelengths, ground_truth, normalize=False)
ground_truth_xyz = ground_truth_xyz / np.min(np.sum(ground_truth_xyz, axis=2))
ground_truth_rgb = from_xyz_to_rgb(ground_truth_xyz)

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(10, 5))
ax0.imshow(ground_truth_rgb)
ax1.imshow(recorded_rgb[::params["downsampling"], ::params["downsampling"]])
ax2.imshow(estimated_rgb)
ax1.set_title("Recorded colors")
ax0.set_title("Original colors")
ax0.set_title("Reconstructed colors")
rect0 = Rectangle((pixel_true[1] - 0.5, pixel_true[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
rect = Rectangle((pixel[1] - 0.5, pixel[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
rect2 = Rectangle((pixel[1] - 0.5, pixel[0] - 0.5), 1, 1, alpha=1, color="none", ec="white", lw=2,zorder=10)
ax1.add_patch(rect)
ax2.add_patch(rect2)
ax0.add_patch(rect0)
plt.savefig(f"{results_path}_low_res.pdf")
plt.show()

### Display spectrum for the pixel marked above

In [ ]:
A = recov.generate_matrix_A(omegas_est, Z, r=params["r"], k0=k0)

fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(6, 2))

show_spectrum(wavelengths, ground_truth[pixel_true], show_background=True, short_display=True, ax=ax0, visible=True)
show_spectrum(wavelengths, downsampled[pixel], show_background=True, short_display=True, ax=ax1, visible=True)
show_spectrum(wavelengths_est, result[pixel], show_background=True, short_display=True, ax=ax2, visible=True)

depths = np.linspace(0,Z,200)
# show_lippmann_transform(depths, lippmann_transform(wavelengths_est, result[pixel], depths, r=params["r"], k0=k0)[0], ax=ax3, short_display=True)

# show_spectrum(wavelengths_est, np.abs(A @ result[pixel])**2, show_background=True, short_display=True, ax=ax4, visible=True)
ax1.set_title("Reflected spectrum")
title = f"Z: {params['Z']}, N: {params['N']}, Visible: {params['visible']}"
if "estimate_depth" in params and params["estimate_depth"]:
    title += f", Z est: {params['Z_estimates'][pixel]:.2e}"
ax2.set_title(title)
# ax3.set_title("Pattern")
# ax4.set_title("Re-estimated reflected spectrum")
# plt.savefig(f"{results_path}_point.pdf")
plt.show()

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(depths, lippmann_transform(wavelengths_est, result[pixel], depths, r=params["r"], k0=k0)[0])
plt.title("Pattern")
plt.gca().axhline(0)
plt.show()

In [ ]:
spectrum = sp.interpolate.interp1d(omegas_rec, downsampled[pixel], kind='cubic', bounds_error=False, 
                                   fill_value='extrapolate')(omegas_est)
plt.figure(figsize=(5,3))
plt.plot(wavelengths_est, spectrum, label="measured", c="k")
plt.plot(wavelengths_est,  np.abs(A @ result[pixel])**2, label="re-estimated")
plt.legend()
plt.gca().axhline()
plt.show()

### Display (and upsample) in RGB

In [ ]:
image = io.imread("Cubes_ours/" + name + ".png")
image = np.swapaxes(image[:, ::-1, :3], 1, 0)
cut_idx = np.loadtxt("Cubes_ours/" + name + "_cut.txt").astype(np.int)
image = image[cut_idx[0, 0]:cut_idx[0, 1], cut_idx[1, 0]:cut_idx[1, 1]]

true_image = io.imread("Cubes_ours/" + name + "_true.png")
true_image = np.swapaxes(true_image[:, ::-1, :3], 1, 0)
true_image = true_image[40:-40, 90:-90]

image = exposure.rescale_intensity(image)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8.5, 5))
ax2.imshow(image)
ax2.axis('off')
ax1.imshow(true_image)
ax1.axis('off')
# plt.savefig("PNAS/color_checker_photos.pdf")
plt.show()

In [ ]:
recorded_upsampled = upsample_hue_saturation(image, recorded_rgb, order=1)
estimated_upsampled = upsample_hue_saturation(image, estimated_rgb, order=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8.5, 5))
ax1.imshow(recorded_upsampled)
ax2.imshow(estimated_upsampled)
ax2.set_title("reconstruced upsampled")
ax1.set_title("measured upsampled")
ax2.axis("off")
ax2.axis("off")
plt.show()